In [4]:

import random
import cirq
import time
def set_io_qubits(qubit_count):
	"""Add the specified number of input and output qubits."""
	
	input_qubits=[cirq.GridQubit(i, 0) for i in range (qubit_count)]
	output_qubit=cirq.GridQubit(qubit_count, 0)
	return (input_qubits, output_qubit)

def make_oracle(input_qubits, output_qubit, x_bits):
	"""Impment function {f(x)=1 if x==x', f(x)=0 if x!=x'}."""
	#make oracle
	#for (1,1) it's just a Toffoli gate
	# other wise negate the zero bits
	
	yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)
	yield(cirq.TOFFOLI(input_qubits[0], input_qubits[1], output_qubit))
	yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)
	

def make_grover_circuit(input_qubits, output_qubit, oracle):
	"""Find the value recognized by the oracle in sqrt(N) attemps. """
	#For 2 input qubits, that means using Grover operator only once.
	c=cirq.Circuit()
	#initialize qubit
	c.append([cirq.X(output_qubit),cirq.H(output_qubit),cirq.H.on_each(*input_qubits),
		   ])
	# Query oracle
	c.append(oracle)
	#Construct Grover operator
	c.append(cirq.H.on_each(*input_qubits))
	c.append(cirq.X.on_each(*input_qubits))
	c.append(cirq.H.on(input_qubits[1]))
	c.append(cirq.CNOT(input_qubits[0], input_qubits[1]))
	c.append(cirq.H.on(input_qubits[1]))
	c.append(cirq.X.on_each(*input_qubits))
	c.append(cirq.H.on_each(*input_qubits))

	#Measure the result
	c.append(cirq.measure(*input_qubits, key='result'))
	return c

def bitstring(bits):
	return ''.join(str(int(b)) for b in bits)
#def idk(input_qubits, output_qubit, x_bits):
#	yield(q for (q, bit) in zip(input_qubits, x_bits) if not bit)

def main():
	start=time.time();
	qubit_count=2
	circuit_sample_count=10
	#set up input and output qubits
	(input_qubits, output_qubit)=set_io_qubits(qubit_count)

	#Choose the x' and make an orcale which can reconginze it
	x_bits=[random.randint(0, 1) for _ in range(qubit_count)]
	print('Secret bit sequence: {}'.format(x_bits))


#	for j in idk(input_qubits, output_qubit, x_bits): 
#		print (j, end = " ") 
	#make oracle (black box)
	oracle=make_oracle(input_qubits, output_qubit, x_bits)
	

	#embed the oracle into a qauntom circuit implmenting Grover's algroithim.
	circuit= make_grover_circuit(input_qubits, output_qubit, oracle)
	print('Circuit:')
	print(circuit)

	#sample from the circuit a couple of times
	simulator=cirq.Simulator()
	result=simulator.run(circuit, repetitions=circuit_sample_count)


	frequencies= result.histogram(key='result', fold_func=bitstring)
	print('Sampled results:\n{}'.format(frequencies))

	#check if we actually found the secret value
	most_common_bitstring=frequencies.most_common(1)[0][0]
	end=time.time();
	print('Most common bitstring: {}'.format(most_common_bitstring))
	print('Found a match: {}'.format(
		most_common_bitstring == bitstring(x_bits)))
	#print('Time it took ')
	#print(end-start)
	

if __name__ == '__main__':
	main()

Secret bit sequence: [0, 1]
Circuit:
(0, 0): ───H───X───@───X───H───X───@───X───H───────M('result')───
                   │               │               │
(1, 0): ───H───────@───H───X───H───X───H───X───H───M─────────────
                   │
(2, 0): ───X───H───X─────────────────────────────────────────────
Sampled results:
Counter({'01': 10})
Most common bitstring: 01
Found a match: True


Grover’s search Algorithm focuses on breaking the limitations of other search algorithms.
When looking through a set of data in terms of finding a piece of that data, we must first have a search protocol. Meaning, we must first know what we must first determine what kind of search we will use and how this search will function. Once we understand the concept behind our search protocol, we must then implement these concepts into an algorithm.
By understanding that we must first have a search protocol rather then an actual algorithm, we come to realize that, to do an exhaustive search algorithm as required to properly search a data base, an analytic approach to our search isn’t possible, so we must use a brute force approach. The reason behind is that, an exhaustive algorithm is to check each possible solution to our problem, which is precisely what a brute force algorithm does.
The problem with doing a search algorithm with a brute force approach is the time complexity behind it. The best run time we can hope for is n/2 steps, if we posit that on average, we can find the target after searching half the range. If we must search the whole range, the run time will be a whole n steps. 
However, the “solution” to this problem would be using a quantum computer. With a quantum computer, we can cut the runtime down to √n. This can be done using Grover’s algorithms, which applies three unitary operates, two of which we implement in a loop until we find our target. The gates used are  the Hadamard (H) gate, the X gate (not gate) and the Toffoli gate (CNOT gate).

Explaining the code in a step by step process
	First we must decide how many time we search the circuit (circuit_sample_count) and the size of our bit string (qubit_count)
	Then we must make our data base (the oracle). 
	To do this, we create the x_bits variable which is representing x’. This is a bit string created of random 0 and 1’s. the x_bits  is the secret variable we are looking for in the database.
	 We then check if the inputput_qubits (which is the x) is equals to the x_bits. If not, then use an x gate on q, negating q inside the circuit (This means the zero bit has been negated).  This is representing the f(x) function. The domain will be all the bit strings that can be made of the length of our qubit_count. The co-domain is just either 0, if the bit string is not equal to our x_bit or 1 if the bit string equals our x_bit.
	For (1, 1) we use the Toffoli gate, negating both bits. Overall, what this is saying is that if the input sequence doesn’t match the secret sequence in some way, its negated. It seems the algorithm is only checking a match 2 bits at a time.
	It seems that when we run our circuit,  the only values we are trying to work with are the values that are not negated. Leading to the next question, What is happening in the step of the circuit involving the oracle. 
    
Now that the database is made, we make the circuit.
	We have to get the output gate as |1> rather then |0> so we append a x gate into our circuit.
	We then append a H gate for each input_qubit and an H gate for our output_qubit.
	After that we can add our database to our circuit, which in this case happens to be a Toffoli gate.
	We then add an H gate for each input_qubits, and an X gate for each input qubit
	We then add an H get for the input qubit at (1, 0).
	We then add a Cnot involving qubits (0,0) and (1,0).
	We then add another H get for the input qubit at (1, 0).
	Again, We then add an H gate for each input_qubits, and an X gate for each input qubit
	Lastly, we append the measurements to get the size of the circuit.
	Since these are yields values, even so the method make_oracle was declared and called first, then method will not actually be called till we do c.append(oracle) inside the make_grover_circuit.
	The next part of the make_grover_circuit, All we do here is construct the operators, which we will run in the stimulator.
	Our oracle is inside the circuit, meaning our database in inside the circuit.
	Our next step is to run through the circuit reaching samples of data.
	We then check to see if our most common bitstring is equal to our secret bit string.
	The reason we are checking the most common is because of the make oracle function. We negate the values that don’t have any matches. Grover algroithim is similar to an inverting function. So in the sense that the values we checked equals our secret bit sequence, its not negated. If it doesn’t, then its negate. The reason we return false if our most common does not match is because it means we did not negate enough bit sequences. (Not sure about this part yet but I believer its say only going through √n was not enough.)
    
How does this impact my career goals.
	I Want to get into artificial intelligence development and one of the challenges of the field will be searching and analyzing data.
	With the understanding of this algorithm, I can perhaps boost the rate of outputs of the ai’s I develop. Depending on the ai I work on, it can have vast and vast of data where n can equal in the millions. By only having to search a square root of that data, I can create much more efficient and faster ai’s.
	Its not an unlikely fact that humans will develop quantum computers in the near future, creating a whole new advancement in technology including the way artificial intelligence can work.
	This will allow me to be a step ahead of the people in my field of study when this piece of technology becomes usable by the common person.
	Computers can go through scenarios and outcomes exponentially faster then any human can. But computers are still under times restraints when making certain decisions. When an ai plays chess, it still limited to seeing all outcomes because it only has 2 minutes to check all possible moves. If we exponential increase its “thinking”, we can in sense make a much more efficient chess opponent. Imagine a chess opponent that can see ALL the possible moves before the first move was even made. Now imagine implementing these kind of ai’s in cars. The possibilities of ai development is endless by itself, but incorporating it with something like grover’s search algorithm, the endless possibilities become even more endless and much more possible.
	The above messaged have all been long term goals, for the near future, being able to understand how grover’s algorithm  works give me the potential to be able to work on other algorithms and understand them using cirq. 

 
Source: Understanding the difference between protocol and algorithm https://www.linkedin.com/pulse/what-difference-between-algorithm-protocol-why-does-matter-acheson
https://algorithmist.com/wiki/Exhaustive_search
A lot of w3 and geeks for geeks for pyhton



Example output:

Secret bit sequence: [1, 1]

Circuit:

(0, 0): ───H───────@───H───X───────@───X───H───────M('result')───

                   │               │               │
                   
(1, 0): ───H───────@───H───X───H───X───H───X───H───M─────────────

                   │

(2, 0): ───X───H───X─────────────────────────────────────────────

Sampled results:

Counter({'11': 10})

Most common bitstring: 11

Found a match: True